In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT *
    FROM nash_nppes
"""

nash_nppes = pd.read_sql(query,db)

In [4]:
db.close()

In [5]:
nash_nppes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38148 entries, 0 to 38147
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   npi                  38148 non-null  int64  
 1   entity_type_code     38148 non-null  float64
 2   org_name             7466 non-null   object 
 3   last_name            30681 non-null  object 
 4   first_name           30682 non-null  object 
 5   middle_name          19790 non-null  object 
 6   name_prefix          11470 non-null  object 
 7   name_suffix          895 non-null    object 
 8   provider_credential  25175 non-null  object 
 9   address_1            38148 non-null  object 
 10  address_2            9877 non-null   object 
 11  city                 38148 non-null  object 
 12  state                38148 non-null  object 
 13  zip                  38148 non-null  object 
 14  taxonomy_code        38148 non-null  object 
 15  cbsa                 38148 non-null 

In [8]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT COUNT(*)
    FROM hop_team
"""

hop_team = pd.read_sql(query,db)

db.close()

In [9]:
hop_team

,COUNT(*)
0,31704890


### How many records in the hop_team table overlap with NPI numbers in nash_nppes?

In [6]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT COUNT(*)
    FROM hop_team
    WHERE from_npi IN(
        SELECT DISTINCT(npi)
        FROM nash_nppes)
    AND to_npi IN(
        SELECT DISTINCT(npi)
        FROM nash_nppes)
"""

hops = pd.read_sql(query,db)

db.close()

In [7]:
hops

,COUNT(*)
0,138493


There are 214,316 records in hop_team where the 'from_npi' NPI code matches one in the nash_nppes table

There are 214,080 records in hop_team where the 'to_npi' NPI code matches one in the nash_nppes table

In total, there are 138,493 hop_team records where both the 'to' and 'from' NPIs are in the nash_nppes table

### Add the filtered table to the database as 'nash_hops'

In [8]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT *
    FROM hop_team
    WHERE from_npi IN(
        SELECT DISTINCT(npi)
        FROM nash_nppes)
    AND to_npi IN(
        SELECT DISTINCT(npi)
        FROM nash_nppes)
"""

nash_hops = pd.read_sql(query,db)

db.close()

In [9]:
nash_hops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138493 entries, 0 to 138492
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   from_npi           138493 non-null  int64  
 1   to_npi             138493 non-null  int64  
 2   patient_count      138493 non-null  int64  
 3   transaction_count  138493 non-null  int64  
 4   average_day_wait   138493 non-null  float64
 5   std_day_wait       138493 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 6.3 MB


In [10]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #open connection

nash_hops.to_sql('nash_hops', db, if_exists = 'append', index = False) #save nash_hops data

db.close() #close connection

### Examine taxonomy

In [2]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT *
    FROM nucc_taxonomy
"""

taxonomy = pd.read_sql(query,db)

db.close()

In [8]:
taxonomy.grouping.unique()

array(['Group', 'Allopathic & Osteopathic Physicians',
       'Behavioral Health & Social Service Providers',
       'Chiropractic Providers', 'Dental Providers',
       'Dietary & Nutritional Service Providers',
       'Emergency Medical Service Providers',
       'Eye and Vision Services Providers', 'Nursing Service Providers',
       'Nursing Service Related Providers', 'Other Service Providers',
       'Pharmacy Service Providers',
       'Physician Assistants & Advanced Practice Nursing Providers',
       'Podiatric Medicine & Surgery Service Providers',
       'Respiratory, Developmental, Rehabilitative and Restorative Service Providers',
       'Speech, Language and Hearing Service Providers',
       'Student, Health Care',
       'Technologists, Technicians & Other Technical Service Providers',
       'Agencies', 'Ambulatory Health Care Facilities', 'Hospital Units',
       'Hospitals', 'Laboratories', 'Managed Care Organizations',
       'Nursing & Custodial Care Facilities',


Exclude these? 

Under Grouping: 
        Laboratories, 
        Suppliers(? DME, Pharmacies, Prosthetic/Orthotic etc.), 
        Transportation Services,
        Group,
        
?Dental

Under Classification: Pharmacy

In [17]:
taxonomy[taxonomy.grouping=='Dental Providers']

,code,grouping,classification,specialization,definition,effective_date,deactivation_date,last_modified_date,notes,display_name
281,125K00000X,Dental Providers,Advanced Practice Dental Therapist,None,An Advanced Practice Dental Therapist is:<br><...,10/1/2012,None,None,Source: Summarized from Minnesota Statute 150A...,Advanced Practice Dental Therapist
282,126800000X,Dental Providers,Dental Assistant,None,An individual who may or may not have complete...,4/1/2002,None,None,Source: Comprehensive Policy Statement on Dent...,Dental Assistant
283,124Q00000X,Dental Providers,Dental Hygienist,None,An individual who has completed an accredited ...,4/1/2002,None,None,Source: Comprehensive Policy Statement on Dent...,Dental Hygienist
284,126900000X,Dental Providers,Dental Laboratory Technician,None,An individual who has the skill and knowledge ...,4/1/2002,None,None,Source: Comprehensive Policy Statement on Dent...,Dental Laboratory Technician
285,125J00000X,Dental Providers,Dental Therapist,None,A Dental Therapist is an individual who has co...,10/1/2012,None,None,Source: Summarized from Minnesota Statute 150A...,Dental Therapist
286,122300000X,Dental Providers,Dentist,None,A dentist is a person qualified by a doctorate...,4/1/2002,None,None,Source: Council on Dental Education and Licens...,Dentist
287,1223D0001X,Dental Providers,Dentist,Dental Public Health,The science and art of preventing and controll...,4/1/2002,None,None,Source: Council on Dental Education and Licens...,Public Health Dentist
288,1223D0004X,Dental Providers,Dentist,Dentist Anesthesiologist,A dentist who has successfully completed an ac...,4/1/2013,None,None,Source: The American Society of Dentist Anesth...,Dentist Anesthesiologist
289,1223E0200X,Dental Providers,Dentist,Endodontics,The branch of dentistry that is concerned with...,4/1/2002,None,None,Source: Council on Dental Education and Licens...,Endodontist
290,1223G0001X,Dental Providers,Dentist,General Practice,A general dentist is the primary dental care p...,4/1/2002,None,None,Source: Academy of General Dentistry,General Practice Dentistry


In [ ]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
SELECT 
    from_npi,
    
FROM nash_hops
"""

taxonomy = pd.read_sql(query,db)

db.close()